In [1]:
#Importing dependencies
import numpy as np
import string
import random
import re
import tensorflow as tf
import pandas as pd
import tensorflow.keras as keras
import matplotlib.pyplot as plt
import sklearn
from tensorflow.keras.models import Sequential
from numpy import array, argmax, random, take
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import RNN, SimpleRNN, LSTM,  Embedding, RepeatVector
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
%matplotlib inline
#For plotting the matplotlib graphs in notebook

In [2]:
# Data_path="D:/Google Drive/Training/Book/0.Chapters/Chapter12 RNN and LSTM/Datasets"
Data_path='Datasets'

In [3]:
import pandas as pd
column_names = ['word1', 'word2', 'word3']

input_3gram = pd.read_csv(Data_path+ "/3Gram_love_data.txt", delimiter='\t', names=column_names) #Importing csv file with column names
print("shape of data", input_3gram.shape)

shape of data (5351, 3)


In [4]:
print("Few sample records from data \n", input_3gram.sample(10))

Few sample records from data 
       word1  word2   word3
3372   love     to     see
3284   love     to     see
1324   love    her    very
2693   love    the     way
387    hate     to    have
26     hate     to  bother
1881   love     it      if
5032  loved  every  minute
2067   love   with     the
4117   love     it    when


In [5]:
print("\nFrequency of word1 values \n", input_3gram["word1"].value_counts())
print("\nFrequency of word2 values \n", input_3gram["word2"].value_counts())


Frequency of word1 values 
 love      4327
loved      416
hate       400
hated       80
loves       72
lovely      24
loving      24
hates        8
Name: word1, dtype: int64

Frequency of word2 values 
 to         1866
it         1361
the         548
with        240
him         144
you         144
of          136
her         104
for          96
and          88
what         56
is           48
each         40
in           40
nothing      32
ones         32
them         32
me           32
as           24
every        24
affair       16
being        16
my           16
more         16
that         16
going        16
got           8
letter        8
about         8
lost          8
your          8
one           8
song          8
story         8
a             8
most          8
view          8
hearing       8
when          8
makes         8
all           8
thy           8
this          8
on            8
husband       8
man           8
at            8
Name: word2, dtype: int64


In [6]:
"""
Finding our words to create dictionary
Here we find unique values in each column and save each of those values .
Later which we will take the unique value for the entire appened columns
This will be our vocabulary list,which are the unique words in our data file
"""
unique_words = []
for i in list(input_3gram.columns.values):
    for j in pd.unique(input_3gram[i]):
        unique_words.append(j)
unique_words = np.unique(unique_words)


print('Count of unique words overall:', len(unique_words))
print('unique words list:', unique_words)

Count of unique words overall: 139
unique words list: ['a' 'able' 'about' 'admit' 'affair' 'affection' 'all' 'and' 'another'
 'answer' 'as' 'at' 'be' 'because' 'being' 'better' 'between' 'bother'
 'break' 'care' 'cared' 'come' 'concern' 'country' 'cut' 'disappoint' 'do'
 'each' 'every' 'fact' 'feel' 'feeling' 'find' 'first' 'for' 'from' 'get'
 'go' 'god' 'going' 'got' 'hate' 'hated' 'hates' 'have' 'he' 'hear'
 'hearing' 'her' 'here' 'him' 'his' 'husband' 'i' 'idea' 'if' 'in'
 'interrupt' 'is' 'it' 'kind' 'know' 'leave' 'letter' 'life' 'like'
 'listen' 'look' 'lost' 'lot' 'love' 'loved' 'lovely' 'loves' 'loving'
 'make' 'makes' 'man' 'marriage' 'me' 'minute' 'more' 'most' 'much'
 'music' 'my' 'nature' 'neighbor' 'not' 'nothing' 'of' 'on' 'one' 'ones'
 'or' 'other' 'over' 'play' 'respect' 'say' 'see' 'sit' 'smell' 'so'
 'someone' 'song' 'sound' 'story' 'stronger' 'support' 'take' 'talk'
 'tell' 'than' 'that' 'the' 'them' 'they' 'think' 'this' 'thought' 'thy'
 'to' 'too' 'united' 'use' 'v

In [7]:
"""
creating our word:indice pair dictionary and inverse
Here will be creating two dictonary values
word_indices : This contains each words mapped to an unique digit 
indices_words : This contains each digits mapped to a word in the same sequence as word_indices 
"""
word_indices = dict((w, i) for i, w in enumerate(unique_words))
indices_words = dict((i, w) for i, w in enumerate(unique_words))

print("word_indices dictionary \n",word_indices)
print("word_indices.keys \n", word_indices.keys())
print("word_indices.values \n", word_indices.values())
print("\n ########################################\n")
print("indices_words dictionary \n", indices_words)
print("indices_words keys \n",indices_words.keys())
print("indices_words values \n",indices_words.values())

word_indices dictionary 
 {'a': 0, 'able': 1, 'about': 2, 'admit': 3, 'affair': 4, 'affection': 5, 'all': 6, 'and': 7, 'another': 8, 'answer': 9, 'as': 10, 'at': 11, 'be': 12, 'because': 13, 'being': 14, 'better': 15, 'between': 16, 'bother': 17, 'break': 18, 'care': 19, 'cared': 20, 'come': 21, 'concern': 22, 'country': 23, 'cut': 24, 'disappoint': 25, 'do': 26, 'each': 27, 'every': 28, 'fact': 29, 'feel': 30, 'feeling': 31, 'find': 32, 'first': 33, 'for': 34, 'from': 35, 'get': 36, 'go': 37, 'god': 38, 'going': 39, 'got': 40, 'hate': 41, 'hated': 42, 'hates': 43, 'have': 44, 'he': 45, 'hear': 46, 'hearing': 47, 'her': 48, 'here': 49, 'him': 50, 'his': 51, 'husband': 52, 'i': 53, 'idea': 54, 'if': 55, 'in': 56, 'interrupt': 57, 'is': 58, 'it': 59, 'kind': 60, 'know': 61, 'leave': 62, 'letter': 63, 'life': 64, 'like': 65, 'listen': 66, 'look': 67, 'lost': 68, 'lot': 69, 'love': 70, 'loved': 71, 'lovely': 72, 'loves': 73, 'loving': 74, 'make': 75, 'makes': 76, 'man': 77, 'marriage': 78,

In [8]:
### Onehot encoding of word1
word1 = input_3gram['word1'].map(word_indices)
word1_onehot = keras.utils.to_categorical(np.array(word1), num_classes=len(word_indices))
print("word1_onehot shape is ",word1_onehot.shape)

word1_onehot shape is  (5351, 139)


In [9]:
#Lets take example of two different words
print("The word in row 0 is -->"+input_3gram['word1'][0])
print("The one hot encoded version of the word in row 0 is \n",word1_onehot[0])

print("\nThe word in row 500 is --> "+input_3gram['word1'][500])
print("The one hot encoded version of the word in row 500 is \n",word1_onehot[500])

The word in row 0 is -->hate
The one hot encoded version of the word in row 0 is 
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]

The word in row 500 is --> love
The one hot encoded version of the word in row 500 is 
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.

In [10]:
##one hot encoding for word2 and word3 
word2 = input_3gram['word2'].map(word_indices)
word2_onehot = keras.utils.to_categorical(np.array(word2), num_classes=len(word_indices))
print("word2_onehot shape is ",word2_onehot.shape)

word3 = input_3gram['word3'].map(word_indices)
word3_onehot = keras.utils.to_categorical(np.array(word3), num_classes=len(word_indices))
print("word3_onehot shape is ",word3_onehot.shape)

word2_onehot shape is  (5351, 139)
word3_onehot shape is  (5351, 139)


### First ANN Model

In [11]:
ANN_model1 = Sequential()
ANN_model1.add(Dense(10, input_dim=word1_onehot.shape[1], activation='sigmoid'))
ANN_model1.add(Dense(word2_onehot.shape[1] ,activation='softmax'))
ANN_model1.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 10)                1400      
_________________________________________________________________
dense_1 (Dense)              (None, 139)               1529      
Total params: 2,929
Trainable params: 2,929
Non-trainable params: 0
_________________________________________________________________


In [12]:
ANN_model1.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# Train model
history = ANN_model1.fit(word1_onehot, word2_onehot, epochs=20, batch_size=50,  verbose=1)

Train on 5351 samples
Epoch 1/20
5351/5351 [==============================] - 1s 131us/sample - loss: 0.0391 - accuracy: 0.9928
Epoch 2/20
5351/5351 [==============================] - 0s 22us/sample - loss: 0.0321 - accuracy: 0.9928
Epoch 3/20
5351/5351 [==============================] - 0s 22us/sample - loss: 0.0265 - accuracy: 0.9928
Epoch 4/20
5351/5351 [==============================] - 0s 22us/sample - loss: 0.0238 - accuracy: 0.9928
Epoch 5/20
5351/5351 [==============================] - 0s 22us/sample - loss: 0.0231 - accuracy: 0.9928
Epoch 6/20
5351/5351 [==============================] - 0s 23us/sample - loss: 0.0228 - accuracy: 0.9928
Epoch 7/20
5351/5351 [==============================] - 0s 21us/sample - loss: 0.0227 - accuracy: 0.9928
Epoch 8/20
5351/5351 [==============================] - 0s 22us/sample - loss: 0.0226 - accuracy: 0.9928
Epoch 9/20
5351/5351 [==============================] - 0s 22us/sample - loss: 0.0225 - accuracy: 0.9928
Epoch 10/20
5351/5351 [=========

In [13]:
#We will see what the 1st hidden layer output representation of the data  
# to predict the hidden layer activations, 
# let's rewrite first layer of our model and give it the weights from fully trained previous model
model1_hidden = Sequential()
model1_hidden.add(Dense(10, input_dim=word1_onehot.shape[1], weights=ANN_model1.layers[0].get_weights()))
model1_hidden.add(Activation('sigmoid'))

In [14]:
# Getting the hidden layer activations
model1_hidden_output = model1_hidden.predict(word1_onehot)
#peak into our hidden layer activations
print("The hidden layer output for every record - Shape of it \n", model1_hidden_output.shape)
print("Few five records from hidden layer \n",model1_hidden_output[:5])

The hidden layer output for every record - Shape of it 
 (5351, 10)
Few five records from hidden layer 
 [[0.8502294  0.7549435  0.8281398  0.81124496 0.7960458  0.803674
  0.8720081  0.66531056 0.8824079  0.8776932 ]
 [0.8502294  0.7549435  0.8281398  0.81124496 0.7960458  0.803674
  0.8720081  0.66531056 0.8824079  0.8776932 ]
 [0.8502294  0.7549435  0.8281398  0.81124496 0.7960458  0.803674
  0.8720081  0.66531056 0.8824079  0.8776932 ]
 [0.8502294  0.7549435  0.8281398  0.81124496 0.7960458  0.803674
  0.8720081  0.66531056 0.8824079  0.8776932 ]
 [0.8502294  0.7549435  0.8281398  0.81124496 0.7960458  0.803674
  0.8720081  0.66531056 0.8824079  0.8776932 ]]


In [15]:
"""
We append the input words of the words2 column in the output of the h1 layer,this gives us the combined input representation
"""
word2_hidden_append = np.append(model1_hidden_output,word2_onehot, axis=1)
print("word2_hidden_append Shape", word2_hidden_append.shape)

word2_hidden_append Shape (5351, 149)


In [16]:
ANN_model2 = Sequential()
ANN_model2.add(Dense(10, input_dim=word2_hidden_append.shape[1], activation='sigmoid'))
ANN_model2.add(Dense(word3_onehot.shape[1], activation='softmax'))
ANN_model2.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 10)                1500      
_________________________________________________________________
dense_4 (Dense)              (None, 139)               1529      
Total params: 3,029
Trainable params: 3,029
Non-trainable params: 0
_________________________________________________________________


In [17]:
ANN_model2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# Train model
history = ANN_model2.fit(word2_hidden_append, word3_onehot, epochs=20, batch_size=50,  verbose=1)

Train on 5351 samples
Epoch 1/20
5351/5351 [==============================] - 0s 92us/sample - loss: 0.0401 - accuracy: 0.9928
Epoch 2/20
5351/5351 [==============================] - 0s 22us/sample - loss: 0.0344 - accuracy: 0.9928
Epoch 3/20
5351/5351 [==============================] - 0s 22us/sample - loss: 0.0309 - accuracy: 0.9928
Epoch 4/20
5351/5351 [==============================] - 0s 22us/sample - loss: 0.0302 - accuracy: 0.9928
Epoch 5/20
5351/5351 [==============================] - 0s 22us/sample - loss: 0.0301 - accuracy: 0.9928
Epoch 6/20
5351/5351 [==============================] - 0s 23us/sample - loss: 0.0299 - accuracy: 0.9928
Epoch 7/20
5351/5351 [==============================] - 0s 22us/sample - loss: 0.0296 - accuracy: 0.9928
Epoch 8/20
5351/5351 [==============================] - 0s 22us/sample - loss: 0.0292 - accuracy: 0.9928
Epoch 9/20
5351/5351 [==============================] - 0s 23us/sample - loss: 0.0287 - accuracy: 0.9928
Epoch 10/20
5351/5351 [==========

In [18]:
# A predict function that takes input word1 and word2; and predict word3 
#1. take the input word , and represent them using digits from the word_indices dictonary values
#2. getting the intermediate hidden nodes for word1
#3. appending hidden activations with word2 as final test set
#4. prediction on this test set
def two_step_pred(words_in):

    index_input=word_indices[words_in[0]]
    indices_in = keras.utils.to_categorical(index_input, num_classes=len(word_indices))
    indices_in=indices_in.reshape(1,len(word_indices))
    h1_test = model1_hidden.predict(indices_in) # getting our intermediate hidden activations from model1h
    
    
    index_input2=word_indices[words_in[1]]
    indices_in2 = keras.utils.to_categorical(index_input2, num_classes=len(word_indices))
    indices_in2= indices_in2.reshape(1,len(word_indices))
    X2_test = np.append(h1_test, indices_in2, axis=1) #preparing final test data by appending hidden with word2
    
    yhat = ANN_model2.predict_classes(X2_test) #predicting final output from model2
    
    print("Input words --> ", words_in)
    print("Predicted word --> ", indices_words[yhat[0]])

In [19]:
two_step_pred(['love', 'it'])
two_step_pred(['love', 'to'])
two_step_pred(['love', 'the'])

Input words -->  ['love', 'it']
Predicted word -->  when
Input words -->  ['love', 'to']
Predicted word -->  see
Input words -->  ['love', 'the']
Predicted word -->  way


## RNNs

In [20]:
model = Sequential()
model.add(SimpleRNN(4, use_bias=False, input_shape=(2,2)))
model.add(Dense(3, use_bias=False, activation='softmax'))
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn (SimpleRNN)       (None, 4)                 24        
_________________________________________________________________
dense_5 (Dense)              (None, 3)                 12        
Total params: 36
Trainable params: 36
Non-trainable params: 0
_________________________________________________________________


In [21]:
model = Sequential()
model.add(SimpleRNN(4, input_shape=(2,2)))
model.add(Dense(3, activation='softmax'))
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_1 (SimpleRNN)     (None, 4)                 28        
_________________________________________________________________
dense_6 (Dense)              (None, 3)                 15        
Total params: 43
Trainable params: 43
Non-trainable params: 0
_________________________________________________________________


In [22]:
model = Sequential()
model.add(SimpleRNN(4, input_shape=(4,2)))
model.add(Dense(3, activation='softmax'))
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_2 (SimpleRNN)     (None, 4)                 28        
_________________________________________________________________
dense_7 (Dense)              (None, 3)                 15        
Total params: 43
Trainable params: 43
Non-trainable params: 0
_________________________________________________________________


## Word prediction using RNN model

In [23]:
word1_word2 = input_3gram[['word1','word2']]
for i in list(word1_word2.columns.values):
    word1_word2[i] = word1_word2[i].map(word_indices)

word1_word2=np.array(word1_word2)
#The same data is reshaped with similar structure but appended with 1 value to make it 3d array
word1_word2=np.reshape(word1_word2,(word1_word2.shape[0],2,1))
word1_word2_onehot = keras.utils.to_categorical(np.array(word1_word2), num_classes=len(word_indices))
print("word1_word2_onehot shape", word1_word2_onehot.shape)

word1_word2_onehot shape (5351, 2, 139)


/Users/shuang/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [24]:
print("time steps" , word1_word2_onehot.shape[1])
print("Input nodes" , word1_word2_onehot.shape[2])
print("output nodes" , word3_onehot.shape[1])

time steps 2
Input nodes 139
output nodes 139


In [25]:
model_rnn = Sequential()
#model.add(SimpleRNN('number of hidden nodes in each rnn cell', input_shape=(timesteps, input_data_dim)))
model_rnn.add(SimpleRNN(30, input_shape=(word1_word2_onehot.shape[1],word1_word2_onehot.shape[2]))) 
model_rnn.add(Dense(word3_onehot.shape[1], activation='softmax'))
model_rnn.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_3 (SimpleRNN)     (None, 30)                5100      
_________________________________________________________________
dense_8 (Dense)              (None, 139)               4309      
Total params: 9,409
Trainable params: 9,409
Non-trainable params: 0
_________________________________________________________________


In [26]:
# compile network
model_rnn.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit network
model_rnn.fit(word1_word2_onehot, word3_onehot, epochs=20)

Train on 5351 samples
Epoch 1/20
5351/5351 [==============================] - 1s 189us/sample - loss: 3.7526 - accuracy: 0.4016
Epoch 2/20
5351/5351 [==============================] - 0s 46us/sample - loss: 2.7391 - accuracy: 0.4947
Epoch 3/20
5351/5351 [==============================] - 0s 47us/sample - loss: 2.3301 - accuracy: 0.5352
Epoch 4/20
5351/5351 [==============================] - 0s 47us/sample - loss: 2.0661 - accuracy: 0.5666
Epoch 5/20
5351/5351 [==============================] - 0s 45us/sample - loss: 1.8881 - accuracy: 0.5842
Epoch 6/20
5351/5351 [==============================] - 0s 46us/sample - loss: 1.7518 - accuracy: 0.6031
Epoch 7/20
5351/5351 [==============================] - 0s 47us/sample - loss: 1.6412 - accuracy: 0.6227
Epoch 8/20
5351/5351 [==============================] - 0s 45us/sample - loss: 1.5513 - accuracy: 0.6416
Epoch 9/20
5351/5351 [==============================] - 0s 46us/sample - loss: 1.4775 - accuracy: 0.6515
Epoch 10/20
5351/5351 [=========

In [27]:
def rnn_word_pred(in_text):
    print("Input is - " , in_text)
    encoded = [word_indices[i] for i in in_text]
    encoded = np.array(encoded).reshape(1,2,1)
    encoded =keras.utils.to_categorical(np.array(encoded), num_classes=len(word_indices))
    ypred = model_rnn.predict_classes(encoded, verbose=0)[0]
    print("Output is --> " ,indices_words[ypred])

In [28]:
rnn_word_pred(['love', 'it'])
rnn_word_pred(['love', 'to'])
rnn_word_pred(['love', 'the'])

Input is -  ['love', 'it']
Output is -->  when
Input is -  ['love', 'to']
Output is -->  see
Input is -  ['love', 'the']
Output is -->  way


## RNN for Long Sequences

In [30]:
longseq_3gram = open(Data_path+'/Long_sequence_3gram.csv').read().lower()
print(longseq_3gram[495:801])
print(longseq_3gram[30615:31000])


a,combination,of
a,combination,of
a,combination,of
a,combination,of
a,combination,of
a,combination,of
a,combination,of
a,combination,of
a,combination,of
a,combination,of
a,combination,of
a,combination,of
a,combination,of
a,combination,of
a,combination,of
a,combination,of
a,combination,of
a,combination,of

and,according,to
and,according,to
and,according,to
and,according,to
and,according,to
and,according,to
and,according,to
and,according,to
and,according,to
and,according,to
and,addresses,of
and,adherence,to
and,advocates,for
and,aerospace,engineering
and,americans,do
and,analyzing,the
and,announced,he
and,announced,he
and,announced,plans
and,announced,that
and,announced,that
and,annou


In [31]:
#Replace comma with space
longseq_3gram1= longseq_3gram.replace(',',' ').replace('\r','')
print(longseq_3gram1[495:750])
print(longseq_3gram1[30615:30800])


a combination of
a combination of
a combination of
a combination of
a combination of
a combination of
a combination of
a combination of
a combination of
a combination of
a combination of
a combination of
a combination of
a combination of
a combination of

and according to
and according to
and according to
and according to
and according to
and according to
and according to
and according to
and according to
and according to
and addresses 


In [32]:
#Unique characters in our dataset we then sort it
chars = sorted(list(set(longseq_3gram1)))
print("Unique Characters in the text \n ",chars)
#\n is character string for new line, we dont need that in our dictionary of chars
chars.remove('\n')
print("\n Character after removing newline symbol \'/n\'",chars)
print("\n overall chars count", len(chars))

Unique Characters in the text 
  ['\n', ' ', "'", '(', '-', '.', '/', '0', '1', '3', '7', '9', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']

 Character after removing newline symbol '\n' [' ', "'", '(', '-', '.', '/', '0', '1', '3', '7', '9', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']

 overall chars count 37


In [33]:
char_indices = dict((c, i) for i, c in enumerate(chars))
print("characters to indices dictionary\n", char_indices)
indices_char = dict((i, c) for i, c in enumerate(chars))
print("indices to char dictionary\n", indices_char)
print('unique chars: ', {len(chars)})

characters to indices dictionary
 {' ': 0, "'": 1, '(': 2, '-': 3, '.': 4, '/': 5, '0': 6, '1': 7, '3': 8, '7': 9, '9': 10, 'a': 11, 'b': 12, 'c': 13, 'd': 14, 'e': 15, 'f': 16, 'g': 17, 'h': 18, 'i': 19, 'j': 20, 'k': 21, 'l': 22, 'm': 23, 'n': 24, 'o': 25, 'p': 26, 'q': 27, 'r': 28, 's': 29, 't': 30, 'u': 31, 'v': 32, 'w': 33, 'x': 34, 'y': 35, 'z': 36}
indices to char dictionary
 {0: ' ', 1: "'", 2: '(', 3: '-', 4: '.', 5: '/', 6: '0', 7: '1', 8: '3', 9: '7', 10: '9', 11: 'a', 12: 'b', 13: 'c', 14: 'd', 15: 'e', 16: 'f', 17: 'g', 18: 'h', 19: 'i', 20: 'j', 21: 'k', 22: 'l', 23: 'm', 24: 'n', 25: 'o', 26: 'p', 27: 'q', 28: 'r', 29: 's', 30: 't', 31: 'u', 32: 'v', 33: 'w', 34: 'x', 35: 'y', 36: 'z'}
unique chars:  {37}


### Char to index on full data

In [34]:
data = longseq_3gram1.splitlines()
##Adding a space at the end
data = [i+' ' for i in data]

##mapping our data into numbers
sentences = [[char_indices[j] for j in i] for i in data ]
print(data[0], sentences[0])
print(data[10], sentences[1])
print(data[20], sentences[2])
print(data[100], sentences[3])
print(data[400], sentences[400])
print(data[4000], sentences[4000])
print(data[9000], sentences[9000])
##Number of sentences
print("Number of sentences ", len(sentences))

a bewildering array  [11, 0, 12, 15, 33, 19, 22, 14, 15, 28, 19, 24, 17, 0, 11, 28, 28, 11, 35, 0]
a celebration of  [11, 0, 12, 15, 24, 15, 16, 19, 13, 19, 11, 28, 35, 0, 25, 16, 0]
a co-director of  [11, 0, 12, 15, 33, 19, 22, 14, 15, 28, 19, 24, 17, 0, 32, 11, 28, 19, 15, 30, 35, 0]
a declaration of  [11, 0, 12, 19, 30, 30, 15, 28, 29, 33, 15, 15, 30, 0, 23, 25, 23, 15, 24, 30, 0]
a significant risk  [11, 0, 29, 19, 17, 24, 19, 16, 19, 13, 11, 24, 30, 0, 28, 19, 29, 21, 0]
been designed as  [12, 15, 15, 24, 0, 14, 15, 29, 19, 17, 24, 15, 14, 0, 11, 29, 0]
from anywhere on  [16, 28, 25, 23, 0, 11, 24, 35, 33, 18, 15, 28, 15, 0, 25, 24, 0]
Number of sentences  30307


### Converting the sentences to RNN friendly data which can be used to generate new sequences

**Take one sentence, iterate through it till the length of sentence is reached:**

* **Step 1** 0:0+14: X; and 14th position: y >> observation 1
* **Step 2** 1:14: X; and 15th position: y >> observation 2
* **Step 3** …we do this till the length of sentence is reached

Take next sentence and repeat the same


In [35]:
#Since all the sentences may not be of same length,it is neccessary to make them consistent when passing to keras
#We select a sequence length
Seq_ln = 14
X = []
y = []
for i in sentences:
    for j in range(len(i)-Seq_ln):
        X.append(i[j:j+Seq_ln])
        y.append(i[j+Seq_ln])
len(X), len(y)

(142142, 142142)

### Sample

In [36]:
print("data[0:2]=", data[0:2])
print("sentences[0:2]=", sentences[0:2])

for i in range (0,20):
    print("X[",i,"]=", X[i],"y[",i,"]=", y[i])

data[0:2]= ['a bewildering array ', 'a beneficiary of ']
sentences[0:2]= [[11, 0, 12, 15, 33, 19, 22, 14, 15, 28, 19, 24, 17, 0, 11, 28, 28, 11, 35, 0], [11, 0, 12, 15, 24, 15, 16, 19, 13, 19, 11, 28, 35, 0, 25, 16, 0]]
X[ 0 ]= [11, 0, 12, 15, 33, 19, 22, 14, 15, 28, 19, 24, 17, 0] y[ 0 ]= 11
X[ 1 ]= [0, 12, 15, 33, 19, 22, 14, 15, 28, 19, 24, 17, 0, 11] y[ 1 ]= 28
X[ 2 ]= [12, 15, 33, 19, 22, 14, 15, 28, 19, 24, 17, 0, 11, 28] y[ 2 ]= 28
X[ 3 ]= [15, 33, 19, 22, 14, 15, 28, 19, 24, 17, 0, 11, 28, 28] y[ 3 ]= 11
X[ 4 ]= [33, 19, 22, 14, 15, 28, 19, 24, 17, 0, 11, 28, 28, 11] y[ 4 ]= 35
X[ 5 ]= [19, 22, 14, 15, 28, 19, 24, 17, 0, 11, 28, 28, 11, 35] y[ 5 ]= 0
X[ 6 ]= [11, 0, 12, 15, 24, 15, 16, 19, 13, 19, 11, 28, 35, 0] y[ 6 ]= 25
X[ 7 ]= [0, 12, 15, 24, 15, 16, 19, 13, 19, 11, 28, 35, 0, 25] y[ 7 ]= 16
X[ 8 ]= [12, 15, 24, 15, 16, 19, 13, 19, 11, 28, 35, 0, 25, 16] y[ 8 ]= 0
X[ 9 ]= [11, 0, 12, 15, 33, 19, 22, 14, 15, 28, 19, 24, 17, 0] y[ 9 ]= 32
X[ 10 ]= [0, 12, 15, 33, 19, 22, 14, 

In [37]:
#The first row is the X's first row up to 14 character
#The second row is the X's first row starting from second character up to 14 character
#The third row is the X's first row starting from third character up to 14 character and so on 
X=np.array(X)
X1=np.reshape(X,(X.shape[0],X.shape[1],1))
X1=keras.utils.to_categorical(np.array(X1), num_classes=len(char_indices))
print(X1.shape)

(142142, 14, 37)


In [38]:
#Target Variable
y[:10]
#Reshapig our label for model
y1 = np.array(y)
# one hot encode outputs
y1 = keras.utils.to_categorical(np.array(y), num_classes=len(char_indices))
y1.shape

(142142, 37)

In [39]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X1, y1, test_size=0.20)
print("X_train.shape", X_train.shape)
print("y_train.shape", y_train.shape)
print("X_test.shape", X_test.shape)
print("y_test.shape", y_test.shape)

X_train.shape (113713, 14, 37)
y_train.shape (113713, 37)
X_test.shape (28429, 14, 37)
y_test.shape (28429, 37)


In [40]:
#building the model
model_RNN2 = Sequential()
##model.add(SimpleRNN('number of hidden nodes in each rnn cell', input_shape=(timesteps, data_dim)))
model_RNN2.add(SimpleRNN(16, input_shape=(X_train.shape[1], X_train.shape[2]))) 
model_RNN2.add(Dense(len(char_indices)))
model_RNN2.add(Activation('softmax'))
model_RNN2.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_4 (SimpleRNN)     (None, 16)                864       
_________________________________________________________________
dense_9 (Dense)              (None, 37)                629       
_________________________________________________________________
activation_1 (Activation)    (None, 37)                0         
Total params: 1,493
Trainable params: 1,493
Non-trainable params: 0
_________________________________________________________________


In [41]:
# compile network
model_RNN2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit network
model_RNN2.fit(X_train, y_train, epochs=30, verbose=1, validation_data=(X_test, y_test))
model_RNN2.save_weights("char_rnn_model_weights_v1.hdf5")

Train on 113713 samples, validate on 28429 samples
Epoch 1/30
113713/113713 [==============================] - 15s 132us/sample - loss: 2.2222 - accuracy: 0.3708 - val_loss: 1.9541 - val_accuracy: 0.4312
Epoch 2/30
113713/113713 [==============================] - 14s 126us/sample - loss: 1.9027 - accuracy: 0.4393 - val_loss: 1.8562 - val_accuracy: 0.4473
Epoch 3/30
113713/113713 [==============================] - 14s 126us/sample - loss: 1.8287 - accuracy: 0.4569 - val_loss: 1.8053 - val_accuracy: 0.4614
Epoch 4/30
113713/113713 [==============================] - 13s 113us/sample - loss: 1.7897 - accuracy: 0.4670 - val_loss: 1.7815 - val_accuracy: 0.4679
Epoch 5/30
113713/113713 [==============================] - 13s 111us/sample - loss: 1.7659 - accuracy: 0.4760 - val_loss: 1.7617 - val_accuracy: 0.4778
Epoch 6/30
113713/113713 [==============================] - 13s 112us/sample - loss: 1.7487 - accuracy: 0.4808 - val_loss: 1.7478 - val_accuracy: 0.4876
Epoch 7/30
113713/113713 [=====

In [42]:
weightsfile_model_RNN2= "char_rnn_model_weights_v1.hdf5"
model_RNN2.load_weights(weightsfile_model_RNN2)

# compile network
model_RNN2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit network
model_RNN2.fit(X_train, y_train, epochs=2, verbose=1)

Train on 113713 samples
Epoch 1/2
113713/113713 [==============================] - 13s 118us/sample - loss: 1.6408 - accuracy: 0.5097
Epoch 2/2
113713/113713 [==============================] - 13s 115us/sample - loss: 1.6395 - accuracy: 0.5094


In [43]:
#function to prepare test input
def prepare_input(in_text):
    X1 = np.array([char_indices[i] for i in in_text]).reshape(1,14,1)
    X1=keras.utils.to_categorical(np.array(X1), num_classes=len(char_indices))
    return(X1)
#function to loop our preditions
def complete_pred(in_text):
    #original_text = in_text
    #generated = in_text
    completion = ''
    while True:
        x = prepare_input(in_text)
        pred = model_RNN2.predict_classes(x, verbose=0)[0]

        next_char = indices_char[pred]

        in_text = in_text[1:] + next_char
        completion += next_char

        if len(completion)> 20 or next_char == ' ':
            return completion

In [44]:
in_text = 'officials say '
out_word = complete_pred(in_text)
print("Input text -->", in_text, "\npredicted word ---> ", out_word)
in_text = 'how dangerous '
out_word = complete_pred(in_text)
print("Input text -->", in_text, "\npredicted output ---> ", out_word)
in_text = 'political and '
out_word = complete_pred(in_text)
print("Input text -->", in_text, "\npredicted output ---> ", out_word)
in_text = 'whatever they '
out_word = complete_pred(in_text)
print("Input text -->", in_text, "\npredicted output ---> ", out_word)
in_text = 'of particular '
out_word = complete_pred(in_text)
print("Input text -->", in_text, "\npredicted output ---> ", out_word)

Input text --> officials say  
predicted word --->  of 
Input text --> how dangerous  
predicted output --->  to 
Input text --> political and  
predicted output --->  the 
Input text --> whatever they  
predicted output --->  of 
Input text --> of particular  
predicted output --->  to 


## LSTM Models

In [45]:
#building the model
model_LSTM = Sequential()
#model1.add(LSTM('number of hidden nodes in each rnn cell', input_shape=(timesteps, data_dim)))
model_LSTM.add(LSTM(128, input_shape=(X_train.shape[1], X_train.shape[2]))) 
model_LSTM.add(Dense(len(char_indices)))
model_LSTM.add(Activation('softmax'))
model_LSTM.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 128)               84992     
_________________________________________________________________
dense_10 (Dense)             (None, 37)                4773      
_________________________________________________________________
activation_2 (Activation)    (None, 37)                0         
Total params: 89,765
Trainable params: 89,765
Non-trainable params: 0
_________________________________________________________________


In [46]:
# compile network
model_LSTM.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit network
model_LSTM.fit(X_train, y_train, epochs=30, verbose=1)
model_LSTM.save_weights("char_LSTM_model_weights_v1.hdf5")

Train on 113713 samples
Epoch 1/30
113713/113713 [==============================] - 36s 319us/sample - loss: 1.9728 - accuracy: 0.4258
Epoch 2/30
113713/113713 [==============================] - 34s 301us/sample - loss: 1.5768 - accuracy: 0.5282
Epoch 3/30
113713/113713 [==============================] - 34s 302us/sample - loss: 1.4047 - accuracy: 0.5811
Epoch 4/30
113713/113713 [==============================] - 34s 301us/sample - loss: 1.2852 - accuracy: 0.6165
Epoch 5/30
113713/113713 [==============================] - 36s 316us/sample - loss: 1.1959 - accuracy: 0.6431
Epoch 6/30
113713/113713 [==============================] - 37s 327us/sample - loss: 1.1253 - accuracy: 0.6646
Epoch 7/30
113713/113713 [==============================] - 34s 303us/sample - loss: 1.0685 - accuracy: 0.6810
Epoch 8/30
113713/113713 [==============================] - 33s 293us/sample - loss: 1.0198 - accuracy: 0.6954
Epoch 9/30
113713/113713 [==============================] - 37s 330us/sample - loss: 0.9

In [48]:
weightsfile_model_LSTM= "char_LSTM_model_weights_v1.hdf5"
model_LSTM.load_weights( weightsfile_model_LSTM)

# compile network
model_LSTM.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit network
model_LSTM.fit(X_train, y_train,epochs=2, verbose=1)

OSError: Unable to open file (unable to open file: name = 'Datasets/Pre_trained_models/char_LSTM_model_weights_v1.hdf5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [ ]:
#function to prepare test input
def prepare_input1(in_text):
    X1 = np.array([char_indices[i] for i in in_text]).reshape(1,14,1)
    X1= keras.utils.to_categorical(np.array(X1), num_classes=len(char_indices))
    return(X1)
#function to loop our preditions
def complete_pred1(in_text):
    #original_text = in_text
    #generated = in_text
    completion = ''
    while True:
        x = prepare_input1(in_text)
        pred = model_LSTM.predict_classes(x, verbose=0)[0]
        next_char = indices_char[pred]

        in_text = in_text[1:] + next_char
        completion += next_char

        if len(completion)> 20 or next_char == ' ':
            return completion

In [ ]:
in_text = 'the emergence '
out_word = complete_pred1(in_text)
print("Input text -->", in_text, "; predicted output ---> ", out_word)
in_text = 'officials say '
out_word = complete_pred1(in_text)
print("Input text -->", in_text, "; predicted output ---> ", out_word)
in_text = 'and sentenced '
out_word = complete_pred1(in_text)
print("Input text -->", in_text, "; predicted output ---> ", out_word)
in_text = 'a combination '
out_word = complete_pred1(in_text)
print("Input text -->", in_text, "; predicted output ---> ", out_word)
in_text = 'and according '
out_word = complete_pred1(in_text)
print("Input text -->", in_text, "; predicted output ---> ", out_word)

### Few more predictions and Comparions with Standard RNN

In [ ]:
in_text = 'how dangerous '
out_word = complete_pred1(in_text)
print("Input text -->", in_text, "\nLSTM Prediction ---> ", out_word)
out_word1 = complete_pred(in_text)
print("RNN Prediction ---> ", out_word1)

print("\n")
in_text = 'political and '
out_word = complete_pred1(in_text)
print("Input text -->", in_text, "\nLSTM Prediction ---> ", out_word)
out_word1 = complete_pred(in_text)
print("RNN Prediction ---> ", out_word1)

print("\n")
in_text = 'of particular '
out_word = complete_pred1(in_text)
print("Input text -->", in_text, "\nLSTM Prediction ---> ", out_word)
out_word1 = complete_pred(in_text)
print("RNN Prediction ---> ", out_word1)

print("\n")
in_text = 'whatever they '
out_word = complete_pred1(in_text)
print("Input text -->", in_text, "\nLSTM Prediction ---> ", out_word)
out_word1 = complete_pred(in_text)
print("RNN Prediction ---> ", out_word1)

## Case Study – Language Translation Project 

In [ ]:
raw_data= open(Data_path+"/fra-eng/fra.txt", mode='rt', encoding='utf-8').read()
raw_data=raw_data.strip().split('\n')
raw_data=[i.split('\t') for i in raw_data]
lang1_lang2_data=array(raw_data)
print(lang1_lang2_data)
print("Overall pairs", len(lang1_lang2_data))

In [ ]:
# Remove punctuation
lang1_lang2_data[:,0] = [word.translate(str.maketrans('', '', string.punctuation)) for word in lang1_lang2_data[:,0]]
lang1_lang2_data[:,1] = [word.translate(str.maketrans('', '', string.punctuation)) for word in lang1_lang2_data[:,1]]

print(lang1_lang2_data)

In [ ]:
## convert text to lowercase
for word in range(len(lang1_lang2_data)):
    lang1_lang2_data[word,0] = lang1_lang2_data[word,0].lower()
    lang1_lang2_data[word,1] = lang1_lang2_data[word,1].lower()
print(lang1_lang2_data)

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(lang1_lang2_data[:, 0])
lang1_tokens=tokenizer
lang1_vocab_size = len(lang1_tokens.word_index) + 1
print("lang1_vocab_size", lang1_vocab_size)

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(lang1_lang2_data[:, 1])
lang2_tokens=tokenizer
lang2_vocab_size = len(lang2_tokens.word_index) + 1
print("lang2_vocab_size", lang2_vocab_size)

In [ ]:
# split data into train and test set
train, test = train_test_split(lang1_lang2_data, test_size=0.1, random_state = 44)

In [ ]:
lang1_seq_length=15
lang2_seq_length=15

X_train_seq=lang1_tokens.texts_to_sequences(train[:, 0])
X_train= pad_sequences(X_train_seq,lang1_seq_length,padding='post')

Y_train_seq=lang2_tokens.texts_to_sequences(train[:, 1])
Y_train= pad_sequences(Y_train_seq,lang2_seq_length,padding='post')

X_test_seq=lang1_tokens.texts_to_sequences(test[:, 0])
X_test= pad_sequences(X_test_seq,lang1_seq_length,padding='post')

Y_test_seq=lang2_tokens.texts_to_sequences(test[:, 1])
Y_test= pad_sequences(Y_test_seq,lang2_seq_length,padding='post')

print("X_train.shape", X_train.shape)
print("Y_train.shape",Y_train.shape)
print("X_test.shape",X_test.shape)
print("Y_test.shape", Y_test.shape)

In [ ]:
print("Text data", [train[5, 0]])
print('Numbers sequence', X_train_seq[5])
print('Padded Sequence', X_train[5])

### Model Building

In [ ]:
model = Sequential()
model.add(Embedding(lang1_vocab_size, 256, input_length=lang1_seq_length, mask_zero=True))
model.add(LSTM(128))
model.add(RepeatVector(lang2_seq_length))
model.add(LSTM(128, return_sequences=True))
model.add(Dense(lang2_vocab_size, activation='softmax'))
model.summary()

In [ ]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')
history = model.fit(X_train, Y_train.reshape(Y_train.shape[0], Y_train.shape[1], 1),  epochs=1, verbose=1, batch_size=1024)
model.save_weights('Eng_fra_model.hdf5')

In [ ]:
model.load_weights(Data_path+"/Pre_trained_models/Eng_fra_model.hdf5")

### Prediction

In [ ]:
def one_line_prediction(text1):
    
    def to_lines(text):
          sents = text.strip().split('\n')
          sents = [i.split('\t') for i in sents]
          return sents
    small_input = to_lines(text1)
    small_input = array(small_input)
    
    # Remove punctuation
    small_input[:,0] = [s.translate(str.maketrans('', '', string.punctuation)) for s in small_input[:,0]]
    # convert text to lowercase
    for i in range(len(small_input)):
        small_input[i,0] = small_input[i,0].lower()

    #encode and pad sequences
    small_input_seq=lang1_tokens.texts_to_sequences(small_input[0])
    small_input= pad_sequences(small_input_seq,lang1_seq_length,padding='post')
   

    #Load the model
    #Eng French Model
    #model.load_weights('/content/drive/My Drive/Training/Book/0.Chapters/Chapter12 RNN and LSTM/1.Archives/Eng_fra_model_v2.hdf5')

    pred_seq = model.predict_classes(small_input[0:1].reshape((small_input[0:1].shape[0],small_input[0:1].shape[1])))
    
    def num_to_word(n, tokens):
          for word, index in tokens.word_index.items():
              if index == n:
                  return word
          return None

    Lang2_text = []
    for word_num in pred_seq:
          sing_pred = []
          for i in range(len(word_num)):
                t = num_to_word(word_num[i], lang2_tokens)
                if i > 0:
                    if (t == num_to_word(word_num[i-1], lang2_tokens)) or (t == None):
                        sing_pred.append('')
                    else:
                        sing_pred.append(t)
                else:
                      if(t == None):
                              sing_pred.append('')
                      else:
                              sing_pred.append(t) 
          Lang2_text.append(' '.join(sing_pred))
    return(Lang2_text)

In [ ]:
Input_sentences=["have a great Good day",
                 "Do you speak English",
                 "I do not know your language",
                 "I need help",
                 "Thank you very much",
                 "Where can I get this",
                 "How much does it cost",
                 "Where is the bathroom",
                 "Where is the ATM",
                 "I am a visitor here",
                 "Excuse me",
                 "What do you do for living",
                 "Here is my passport"]

for sent in Input_sentences:
  print([sent] , " -->",one_line_prediction(sent))